In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
viewmetrics = pd.read_pickle('../../geodata/results/00_building_level_view_metrics.pkl')
gdf = gpd.GeoDataFrame(data = viewmetrics, geometry=gpd.points_from_xy(x = viewmetrics.FassPktX,y = viewmetrics.FassPktY), crs = 2056)

In [5]:
#import geodata
ch_borders = gpd.read_file('../../geodata/ch_districts/ch_full.gpkg')
ch_income  = gpd.read_file('../../geodata/ch_districts/ch_income_per_commune_2018.gpkg')
ch_agglo   = gpd.read_file('../../geodata/ch_agglo/swiss_urban_agglomerations.gpkg')
ch_muni    = gpd.read_file('../../geodata/ch_gemeindetypen/ch_gemeindetypen_00_merged.gpkg') 



In [6]:
#income in units of CHF 1,000
ch_income['net_income_ptp']  = (ch_income.net_income_ptp / 10**3).round(2)
ch_income['taxable_income_ptp']  = (ch_income.taxable_income_ptp / 10**3).round(2)

#update agglo names
ch_agglo['Agglo_Name'] = ch_agglo.Agglo_Name.str.replace('\xa0','')

In [7]:
income_cols = ch_income.columns[-8:-1].tolist()

In [8]:
#remove points outside of national boundary

poly_idx, pt_idx = gdf.geometry.sindex.query_bulk(ch_borders.dissolve().geometry, predicate = 'contains')
print(f'Num points outside of national bounday: {len(np.setdiff1d(range(0,gdf.shape[0]), pt_idx))}')

Num points outside of national bounday: 23876


In [9]:
#remove points outside of Lichenstein
poly_idx, foriegn_idx = gdf.geometry.sindex.query_bulk(ch_agglo.query('Land == "LI"').dissolve().geometry, predicate = 'contains')
ch_idx = np.setdiff1d(range(0,gdf.shape[0]), foriegn_idx)
print(f'Num points in Lichenstein: {len(foriegn_idx)}')

Num points in Lichenstein: 20398


In [10]:
# merge income and agglomeration datasets
ch_income_w_agglo = pd.merge(ch_income, ch_agglo[['Gem_No','Name','Agglo_Name']], left_on = 'GMDNR', right_on = 'Gem_No', how = 'left')

In [11]:
#limit points to outside of LI, assume
dat_ = gdf.iloc[ch_idx,:]

In [12]:
%%time

#Spatial Join with Income + Agglo GeoFrame
dat_2 = gpd.sjoin(ch_income_w_agglo[income_cols +['GMDNAME','GMDNR','KTNR','AREA_HA','Name','Agglo_Name','geometry']],
                  dat_,
                  how='right',
                        )
dat_2.drop('index_left', axis=1, inplace=True)

CPU times: total: 19.8 s
Wall time: 19.8 s


In [13]:
%%time

#Spatial Join with Income GeoFrame
missing_income = gpd.sjoin_nearest(ch_income_w_agglo[income_cols +['GMDNAME','GMDNR','KTNR','AREA_HA','Name','Agglo_Name','geometry']],
                          dat_.loc[dat_2.GMDNAME.isnull(),:],
                          how='right',
                          max_distance=1500, distance_col='dist'
                         )

missing_income.drop('index_left', axis=1, inplace=True)
missing_income = missing_income[~missing_income.index.duplicated(keep='first')]
dat_2 = pd.concat([dat_2.loc[~dat_2.GMDNAME.isnull(),:], missing_income], axis = 0)                   

CPU times: total: 7.97 s
Wall time: 7.97 s


In [14]:
%%time
#Spatial Join with TYP00 GeoFrame
dataset = gpd.sjoin(ch_muni[['NAME','TYP_00','geometry']],dat_2,how='right')
dataset.drop('index_left', axis=1, inplace=True)

dataset['Agglo_Name'] = dataset.Agglo_Name.fillna('Rural')
dataset['Rich_Nabr']  = dataset['TYP_00'].map(lambda x: 'R' if x == 'Einkommensstarke Gemeinden (RE)' else 'NR')
dataset['tprsn']      = ((dataset.taxable_income* 10**6) / dataset.taxable_income_ptp).round(0)

CPU times: total: 24.1 s
Wall time: 24.1 s


In [15]:
dataset.info() # option to slim down

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3299410 entries, {000001BB-9927-4513-8016-85310A32D70C} to {FFF90A28-E849-4B5C-97E5-9EC3BE839FC3}
Data columns (total 84 columns):
 #   Column              Dtype   
---  ------              -----   
 0   NAME                object  
 1   TYP_00              object  
 2   net_income          float64 
 3   net_income_ptp      float64 
 4   taxable_income      float64 
 5   taxable_income_ptp  float64 
 6   deductions          float64 
 7   deductions_ptp      float64 
 8   deductPCT           float64 
 9   GMDNAME             object  
 10  GMDNR               float64 
 11  KTNR                float64 
 12  AREA_HA             float64 
 13  Name                object  
 14  Agglo_Name          object  
 15  maxvsh_Abb7         float64 
 16  maxvsh_Abw14        float64 
 17  maxvsh_Dac1         float64 
 18  maxvsh_Fas2         float64 
 19  maxvsh_Flu18        float64 
 20  maxvsh_Geb12        float64 
 21  maxvsh_Gew1         float64 


In [16]:
dataset.to_pickle('../../geodata/results/01_master_building_dataset.pkl')